# Paper 30: Lost in the Middle: How Language Models Use Long Contexts
## Nelson F. Liu, Kevin Lin, John Hewitt, et al., Stanford & UW (2023)

### The "Lost in the Middle" Phenomenon

Language models struggle to use information in the middle of long contexts. Performance follows a U-shaped curve!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## Simulate Multi-Document QA Task

**Setup**: 
- Query requires information from ONE document
- Multiple documents provided (1 relevant, rest distractors)
- **Question**: Does position of relevant document matter?

In [ ]:
class Document:
    def __init__(self, content, is_relevant=False):
        self.content = content
        self.is_relevant = is_relevant
    
    def __repr__(self):
        return f"Doc(relevant={self.is_relevant}): {self.content[:50]}..."

# Create synthetic documents
relevant_doc = Document(
    "The Eiffel Tower was completed in 1889 and stands 330 meters tall. "
    "It was designed by Gustave Eiffel for the 1889 World's Fair in Paris.",
    is_relevant=True
)

distractor_docs = [
    Document("The Great Wall of China is over 13,000 miles long and was built over many centuries."),
    Document("The Statue of Liberty was gifted by France to the United States in 1886."),
    Document("Mount Everest is the tallest mountain on Earth at 8,849 meters above sea level."),
    Document("The Amazon River is the largest river by discharge volume in the world."),
    Document("The Sahara Desert is the largest hot desert, covering much of North Africa."),
    Document("The Colosseum in Rome was completed in 80 AD and could hold 50,000 spectators."),
    Document("The Taj Mahal in India was built between 1632 and 1653 as a mausoleum."),
    Document("The Grand Canyon in Arizona is 277 miles long and up to 18 miles wide."),
    Document("The Great Barrier Reef is the world's largest coral reef system."),
]

query = "When was the Eiffel Tower completed?"
correct_answer = "1889"

print(f"Query: {query}")
print(f"Correct answer: {correct_answer}")
print(f"\nRelevant document: {relevant_doc.content}")
print(f"\nNumber of distractor documents: {len(distractor_docs)}")

## Simplified Language Model

Simulate attention-based model with position bias

In [ ]:
class SimpleLM:
    """Simplified LM with position bias"""
    def __init__(self, position_bias_type='u_shaped'):
        """
        position_bias_type:
        - 'uniform': Equal attention to all positions
        - 'u_shaped': High at beginning/end, low in middle
        - 'recency': Prefer recent (end) positions
        - 'primacy': Prefer early (beginning) positions
        """
        self.position_bias_type = position_bias_type
    
    def get_position_weights(self, num_positions):
        """Compute position-based attention weights"""
        positions = np.arange(num_positions)
        
        if self.position_bias_type == 'uniform':
            weights = np.ones(num_positions)
        
        elif self.position_bias_type == 'u_shaped':
            # U-shaped: high at edges, low in middle
            normalized_pos = positions / (num_positions - 1)  # 0 to 1
            # Quadratic with minimum at 0.5
            weights = 4 * (normalized_pos - 0.5) ** 2 + 0.3
        
        elif self.position_bias_type == 'recency':
            # Exponential decay towards beginning
            weights = np.exp(positions * 0.2)
        
        elif self.position_bias_type == 'primacy':
            # Exponential decay towards end
            weights = np.exp(-positions * 0.2)
        
        # Normalize
        weights = weights / np.sum(weights)
        return weights
    
    def answer_query(self, query, documents):
        """
        Simulate answering query using documents
        Returns: probability of finding correct answer
        """
        num_docs = len(documents)
        
        # Get position weights
        position_weights = self.get_position_weights(num_docs)
        
        # Find relevant document position
        relevant_position = None
        for i, doc in enumerate(documents):
            if doc.is_relevant:
                relevant_position = i
                break
        
        if relevant_position is None:
            return 0.0  # No relevant document
        
        # Probability of using relevant document
        # Higher weight → more likely to use that document
        prob_correct = position_weights[relevant_position]
        
        return prob_correct

# Test different bias types
num_docs = 10
test_positions = np.arange(num_docs)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

bias_types = ['uniform', 'u_shaped', 'recency', 'primacy']
for ax, bias_type in zip(axes, bias_types):
    model = SimpleLM(position_bias_type=bias_type)
    weights = model.get_position_weights(num_docs)
    
    ax.bar(test_positions, weights, color='steelblue', edgecolor='black')
    ax.set_xlabel('Document Position', fontsize=11)
    ax.set_ylabel('Attention Weight', fontsize=11)
    ax.set_title(f'{bias_type.replace("_", " ").title()} Bias', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, max(weights) * 1.2)

plt.tight_layout()
plt.show()

print("\nReal LLMs show U-shaped bias (high at beginning/end, low in middle)!")

## Test Position Sensitivity

In [ ]:
def test_all_positions(model, query, relevant_doc, distractor_docs):
    """
    Test performance with relevant document at each position
    """
    num_positions = len(distractor_docs) + 1
    accuracies = []
    
    for pos in range(num_positions):
        # Create document list with relevant doc at position 'pos'
        docs = distractor_docs[:pos] + [relevant_doc] + distractor_docs[pos:]
        docs = docs[:num_positions]  # Keep fixed length
        
        # Get model's probability of answering correctly
        prob_correct = model.answer_query(query, docs)
        accuracies.append(prob_correct)
    
    return accuracies

# Test U-shaped bias (realistic)
model_realistic = SimpleLM(position_bias_type='u_shaped')
accuracies_realistic = test_all_positions(model_realistic, query, relevant_doc, distractor_docs)

# Test uniform (ideal)
model_ideal = SimpleLM(position_bias_type='uniform')
accuracies_ideal = test_all_positions(model_ideal, query, relevant_doc, distractor_docs)

# Plot
positions = np.arange(len(accuracies_realistic))

plt.figure(figsize=(12, 6))
plt.plot(positions, accuracies_realistic, 'o-', linewidth=3, markersize=10, 
        label='Realistic (U-shaped bias)', color='crimson')
plt.plot(positions, accuracies_ideal, 's--', linewidth=2, markersize=8, 
        label='Ideal (No bias)', color='green', alpha=0.6)

# Mark beginning and end
plt.axvline(x=0, color='blue', linestyle=':', alpha=0.5, linewidth=2, label='Beginning')
plt.axvline(x=len(positions)-1, color='purple', linestyle=':', alpha=0.5, linewidth=2, label='End')

# Mark middle region
middle_start = len(positions) // 4
middle_end = 3 * len(positions) // 4
plt.axvspan(middle_start, middle_end, alpha=0.2, color='red', label='Middle (worst)')

plt.xlabel('Position of Relevant Document', fontsize=13)
plt.ylabel('Accuracy', fontsize=13)
plt.title('Lost in the Middle: Performance vs Position', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Stats
beginning_acc = accuracies_realistic[0]
middle_acc = np.mean(accuracies_realistic[middle_start:middle_end])
end_acc = accuracies_realistic[-1]

print(f"\nPerformance Analysis:")
print(f"Beginning (pos 0): {beginning_acc:.1%}")
print(f"Middle (pos {middle_start}-{middle_end}): {middle_acc:.1%}")
print(f"End (pos {len(positions)-1}): {end_acc:.1%}")
print(f"\nMiddle penalty: -{(beginning_acc - middle_acc)/beginning_acc:.1%} relative to beginning")

## Impact of Context Length

In [ ]:
def test_varying_lengths(model, query, relevant_doc, distractor_docs, lengths):
    """
    Test how performance changes with context length
    """
    results = {'beginning': [], 'middle': [], 'end': []}
    
    for length in lengths:
        # Use subset of distractors
        current_distractors = distractor_docs[:length-1]
        
        # Test three positions: beginning, middle, end
        positions = {
            'beginning': 0,
            'middle': length // 2,
            'end': length - 1
        }
        
        for pos_name, pos in positions.items():
            docs = current_distractors[:pos] + [relevant_doc] + current_distractors[pos:]
            docs = docs[:length]
            
            acc = model.answer_query(query, docs)
            results[pos_name].append(acc)
    
    return results

# Test different context lengths
lengths = [3, 5, 7, 9, 10]
results = test_varying_lengths(model_realistic, query, relevant_doc, distractor_docs, lengths)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(lengths, results['beginning'], 'o-', linewidth=3, markersize=10, 
        label='Beginning', color='blue')
plt.plot(lengths, results['middle'], 's-', linewidth=3, markersize=10, 
        label='Middle', color='red')
plt.plot(lengths, results['end'], '^-', linewidth=3, markersize=10, 
        label='End', color='purple')

plt.xlabel('Number of Documents', fontsize=13)
plt.ylabel('Accuracy', fontsize=13)
plt.title('Performance Degradation with Context Length', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nLonger contexts → worse performance (especially in middle!)")

## Ordering Strategies for RAG

In [ ]:
def order_documents(documents, relevance_scores, strategy='default'):
    """
    Order documents according to strategy
    
    Strategies:
    - 'default': Keep retrieval order
    - 'most_relevant_first': Put best documents at beginning
    - 'most_relevant_edges': Put best at beginning & end
    - 'reverse': Reverse retrieval order
    """
    indices = np.arange(len(documents))
    
    if strategy == 'default':
        return documents
    
    elif strategy == 'most_relevant_first':
        # Sort by relevance (descending)
        sorted_indices = np.argsort(relevance_scores)[::-1]
        return [documents[i] for i in sorted_indices]
    
    elif strategy == 'most_relevant_edges':
        # Put most relevant at beginning and end
        sorted_indices = np.argsort(relevance_scores)[::-1]
        
        # Interleave: best at edges, worst in middle
        ordered = []
        for i in range(len(documents) // 2):
            ordered.append(documents[sorted_indices[i]])  # High relevance
        for i in range(len(documents) // 2, len(documents)):
            ordered.append(documents[sorted_indices[i]])  # Low relevance
        
        # Reverse second half to put high at end
        mid = len(ordered) // 2
        return ordered[:mid] + ordered[mid:][::-1]
    
    elif strategy == 'reverse':
        return documents[::-1]
    
    return documents

# Simulate retrieval scores
num_test_docs = 10
test_docs = [relevant_doc] + distractor_docs[:num_test_docs-1]

# Relevance scores (relevant doc gets high score)
relevance_scores = np.random.rand(num_test_docs) * 0.5
relevance_scores[0] = 0.95  # Relevant doc has high score

# Shuffle to simulate retrieval
shuffle_idx = np.random.permutation(num_test_docs)
test_docs = [test_docs[i] for i in shuffle_idx]
relevance_scores = relevance_scores[shuffle_idx]

# Test different strategies
strategies = ['default', 'most_relevant_first', 'most_relevant_edges']
strategy_accuracies = {}

for strategy in strategies:
    ordered = order_documents(test_docs, relevance_scores, strategy)
    acc = model_realistic.answer_query(query, ordered)
    strategy_accuracies[strategy] = acc
    
    # Find position of relevant doc
    rel_pos = next(i for i, doc in enumerate(ordered) if doc.is_relevant)
    print(f"\n{strategy:25s}: Relevant doc at position {rel_pos:2d}, Accuracy: {acc:.1%}")

# Visualize
plt.figure(figsize=(10, 6))
bars = plt.bar(range(len(strategies)), 
              [strategy_accuracies[s] for s in strategies],
              color=['lightcoral', 'lightblue', 'lightgreen'],
              edgecolor='black', linewidth=2)

plt.xticks(range(len(strategies)), 
          [s.replace('_', '\n').title() for s in strategies],
          fontsize=11)
plt.ylabel('Accuracy', fontsize=13)
plt.title('Document Ordering Strategies', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, strategy in zip(bars, strategies):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{strategy_accuracies[strategy]:.1%}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("RECOMMENDATION: Put most important documents at edges!")
print("="*60)

## Attention Pattern Analysis

In [ ]:
# Simulate attention patterns for different context lengths
context_lengths = [10, 20, 30]
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

for ax, length in zip(axes, context_lengths):
    # Generate attention weights (U-shaped)
    positions = np.arange(length)
    normalized = positions / (length - 1)
    attention = 4 * (normalized - 0.5) ** 2 + 0.3
    attention = attention / np.sum(attention)
    
    # Plot
    ax.bar(positions, attention, color='steelblue', edgecolor='black', linewidth=1)
    ax.set_xlabel('Position', fontsize=11)
    ax.set_ylabel('Attention Weight', fontsize=11)
    ax.set_title(f'Context Length = {length}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Highlight middle region
    middle_start = length // 4
    middle_end = 3 * length // 4
    ax.axvspan(middle_start, middle_end, alpha=0.2, color='red')

plt.suptitle('Attention Patterns: Lost in the Middle', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\nAs context grows, middle positions get even less attention!")

## Key Takeaways

### The Lost in the Middle Phenomenon:

**Observation**: Language models show **U-shaped performance curve**
- ✅ High accuracy when relevant info is at **beginning**
- ✅ High accuracy when relevant info is at **end**  
- ❌ **Low accuracy** when relevant info is in the **middle**

### Why Does This Happen?

**Hypotheses**:

1. **Attention patterns**:
   - Self-attention naturally focuses on recent tokens (recency bias)
   - Also focuses on early tokens (primacy bias)
   - Middle tokens receive less attention

2. **Training distribution**:
   - Most training documents are short
   - Long contexts are rare in pre-training
   - Models haven't learned to use middle well

3. **Causal masking**:
   - Decoder models can't "look ahead"
   - Information in middle may be "overwritten" by later tokens

### Experimental Findings:

**From the paper**:

**Multi-document QA**:
- Relevant doc at position 1 (beginning): ~90% accuracy
- Relevant doc at position 5 (middle): ~60% accuracy  
- Relevant doc at position 10 (end): ~85% accuracy

**Effect of context length**:
- 10 documents: Middle penalty ~30%
- 20 documents: Middle penalty ~40%
- 30 documents: Middle penalty ~50%

**Models tested**:
- GPT-3.5-turbo: Strong U-shaped bias
- Claude: Strong U-shaped bias
- GPT-4: Mitigated but still present
- Open-source LLMs: Even stronger bias

### Position Bias Formula:

Performance at position $p$ (normalized 0-1):
$$
\text{Accuracy}(p) \propto 4(p - 0.5)^2 + c
$$

Where:
- Minimum at $p = 0.5$ (middle)
- Maximum at $p = 0$ and $p = 1$ (edges)
- $c$ is baseline performance

### Implications for RAG Systems:

**Problem**:
```
Retriever returns: [Doc1, Doc2, ..., Doc20]
                    (sorted by relevance score)

If most relevant doc is in middle → poor performance!
```

**Solutions**:

1. **Reorder retrieved documents**:
   - Put most relevant at beginning
   - Or interleave: best at edges, worst in middle

2. **Limit context length**:
   - Use fewer, more relevant documents
   - Top-3 or top-5 instead of top-20

3. **Chunking**:
   - Process long contexts in smaller chunks
   - Aggregate results

4. **Explicit attention**:
   - Fine-tune model to attend to middle
   - Add position embeddings that counter bias

### Document Ordering Strategies:

| Strategy | Description | Performance |
|----------|-------------|-------------|
| Retrieval order | Keep as retrieved | Baseline |
| Most relevant first | Best at beginning | Good |
| Most relevant edges | Best at begin & end | **Best** |
| Reverse | Flip retrieval order | Varies |

### Best Practices:

1. **Short contexts** when possible
2. **Important info at edges** (beginning or end)
3. **Rerank** documents before passing to LLM
4. **Chunk** very long contexts
5. **Test** position sensitivity for your model

### Code Example (Reordering):

```python
def reorder_for_llm(docs, scores):
    """Put most relevant at edges"""
    sorted_idx = np.argsort(scores)[::-1]
    
    # Interleave high and low relevance
    reordered = []
    for i in range(len(docs) // 2):
        reordered.append(docs[sorted_idx[i]])  # High
    for i in range(len(docs) // 2, len(docs)):
        reordered.append(docs[sorted_idx[i]])  # Low
    
    # Move best to end as well
    mid = len(reordered) // 2
    return reordered[:mid] + reordered[mid:][::-1]
```

### Mitigation Strategies:

**During training**:
- Include long-context examples
- Explicitly supervise middle positions
- Use position-aware objectives

**During inference**:
- Reorder documents strategically
- Use multiple passes (process subsets)
- Explicit prompting: "Focus on all documents equally"

**Architecture changes**:
- Sparse attention patterns
- Hierarchical processing
- Retrieval-augmented attention

### Future Directions:

- **Position-invariant models**: Train to ignore position bias
- **Adaptive attention**: Learn to focus on relevant parts
- **Chunked processing**: Process in overlapping windows
- **Multi-pass reasoning**: Multiple reads of context

### Takeaway Message:

```
⚠️  WARNING: Don't assume LLMs use all context equally!

✅  DO: Test position sensitivity
✅  DO: Put important info at edges  
✅  DO: Keep contexts short when possible
❌  DON'T: Assume middle positions work well
❌  DON'T: Blindly concatenate many documents
```

### Impact:

This paper revealed a critical limitation of current LLMs and changed how we think about:
- RAG system design
- Long-context evaluation
- Document ordering for QA
- Prompt engineering with multiple sources

**Remember**: Even with 100k+ context windows, position matters!